In [ ]:
# pip install regex
# pip install emoji
# pip install numpy
# pip install plotly
# pip install matplotlib
# pip install wordcloud

In [ ]:
import pandas as pd
import re

import regex
import emoji

import numpy as np
from collections import Counter

import plotly.express as px
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

# PRIMEROS PASOS, PREPARR DATOS

In [ ]:
with open('./data.txt', encoding='utf-8') as file:
    content = file.read()

print(content)

In [ ]:
# Patron regex para identificar el comienzo de cada línea del txt con la fecha y la hora
def IniciaConFechaYHora(s):
    # Ejemplo: '9/16/23, 5:59 PM - ...'
    patron = re.compile(r'(\d+/\d+/\d+, \d+:\d+ [ap]\. m\.) - ([^:]+): (.+)')
    resultado = re.match(patron, s)  # Verificar si cada línea del txt hace match con el patrón de fecha y hora
    if resultado:
        return True
    return False

# Separar las partes de cada línea del txt: Fecha, Hora, Miembro y Mensaje
def ObtenerPartes(linea):
    # Aplicar el patrón a la línea
    pattern = re.compile(r'(\d+/\d+/\d+, \d+:\d+ [ap]\. m\.) - ([^:]+): (.+)')
    match = pattern.match(linea)

    # Verificar si hay coincidencia
    if match:
        # Obtener las partes deseadas
        date_time = match.group(1)
        Miembro = match.group(2)
        Mensaje = match.group(3)

        # Separar fecha y hora
        Fecha, Hora = date_time.split(', ')

        # Retornar los resultado
        return Fecha, Hora, Miembro, Mensaje
    else:
        return None, None, None, None

# TABLA DE MENSAJES LIMPIOS

In [ ]:
# Leer el archivo txt descargado del chat de WhatsApp
RutaChat = './data.txt'

# Lista para almacenar los datos (Fecha, Hora, Miembro, Mensaje) de cada línea del txt
DatosLista = []
with open(RutaChat, encoding="utf-8") as fp:
    lines = fp.readlines()[1:]  # Eliminar primera fila relacionada al cifrado de extremo a extremo
    print(lines)
    DatosLista = [ObtenerPartes(line.strip()) for line in lines if IniciaConFechaYHora(line.strip())]

df = pd.DataFrame(DatosLista, columns=['Fecha', 'Hora', 'Miembro', 'Mensaje'])
df['Fecha'] = pd.to_datetime(df['Fecha'], format="%d/%m/%Y", errors='coerce')

df = df.dropna().reset_index(drop=True)
pd.DataFrame(df)


# FILTRAR FECHA

In [ ]:
start_date = '2023-07-15'#fecha inicio
end_date = '2023-11-30'#fecha fin

df = df[(df['Fecha'] >= start_date) & (df['Fecha'] <= end_date)]
df

# MENSAJES, IMAGENES, LINK, ENCUESTAS Y EMOJIS

In [ ]:
def ObtenerEmojis(Mensaje):
    emoji_lista = []
    data = regex.findall(r'\X', Mensaje) # Obtener lista de caracteres de cada mensaje
    for caracter in data:
        if caracter in emoji.EMOJI_DATA: # Obtener emojis en idioma español
            emoji_lista.append(caracter)
    return emoji_lista

# Obtener la cantidad total de mensajes
total_mensajes = df.shape[0]

# Obtener la cantidad de archivos multimedia enviados
multimedia_mensajes = df[df['Mensaje'] == '<Multimedia omitido>'].shape[0]

# Obtener la cantidad de emojis enviados
df['Emojis'] = df['Mensaje'].apply(ObtenerEmojis) # Se agrega columna 'Emojis'
emojis = sum(df['Emojis'].str.len())

# Obtener la cantidad de links enviados
url_patron = r'(https?://\S+)'
df['links'] = df.Mensaje.apply(lambda x: len(re.findall(url_patron, x))) # Se agrega columna 'links'
links = sum(df['links'])

# Obtener la cantidad de encuestas
encuestas = df[df['Mensaje'] == 'POLL:'].shape[0]

# Todos los datos pasarlo a diccionario
estadistica_dict = {'Tipo': ['Mensajes', 'Multimedia', 'Emojis', 'Links', 'Encuestas'],
        'Cantidad': [total_mensajes, multimedia_mensajes, emojis, links, encuestas]
        }

#Convertir diccionario a dataframe
estadistica_df = pd.DataFrame(estadistica_dict, columns = ['Tipo', 'Cantidad'])

# Establecer la columna Tipo como índice
estadistica_df = estadistica_df.set_index('Tipo')
estadistica_df

# EMOJIS MAS USADOS

In [ ]:
# Obtener emojis más usados y las cantidades en el chat del grupo del dataframe
emojis_lista = list([a for b in df.Emojis for a in b])
emoji_diccionario = dict(Counter(emojis_lista))
emoji_diccionario = sorted(emoji_diccionario.items(), key=lambda x: x[1], reverse=True)

# Convertir el diccionario a dataframe
emoji_df = pd.DataFrame(emoji_diccionario, columns=['Emoji', 'Cantidad'])

# Establecer la columna Emoji como índice
emoji_df = emoji_df.set_index('Emoji')

print('Número emojis únicos usados: ', len(emoji_df), '\n')
emoji_df.head(10)

In [ ]:
plt.figure(figsize=(10, 8))
plt.pie(emoji_df['Cantidad'], labels=emoji_df.index, autopct='%1.1f%%', startangle=120)
plt.title('Porcentaje de Uso de Emojis en el Chat del Grupo')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

# ESTADISTICAS DE PERSONAS

In [ ]:
# Determinar los miembros más activos del grupo
df_MiembrosActivos = df.groupby('Miembro')['Mensaje'].count().sort_values(ascending=False).to_frame()
df_MiembrosActivos.reset_index(inplace=True)
df_MiembrosActivos.index = np.arange(1, len(df_MiembrosActivos)+1)
df_MiembrosActivos

In [ ]:
# Separar mensajes (sin multimedia) y multimedia (stickers, fotos, videos)
multimedia_df = df[df['Mensaje'] == '<Multimedia omitido>']
mensajes_df = df.drop(multimedia_df.index)

# Contar la cantidad de palabras y letras por mensaje
mensajes_df['Letras'] = mensajes_df['Mensaje'].apply(lambda s : len(s))
mensajes_df['Palabras'] = mensajes_df['Mensaje'].apply(lambda s : len(s.split(' ')))
pd.DataFrame(mensajes_df)


In [ ]:
# Obtener a todos los miembros
miembros = mensajes_df.Miembro.unique()

# Crear diccionario donde se almacenará todos los datos
dictionario = {}

for i in range(len(miembros)):
    lista = []
    # Filtrar mensajes de un miembro en específico
    miembro_df= mensajes_df[mensajes_df['Miembro'] == miembros[i]]

    # Agregar a la lista el número total de mensajes enviados
    lista.append(miembro_df.shape[0])
    
    # Agregar a la lista el número de palabras por total de mensajes (palabras por mensaje)
    palabras_por_msj = (np.sum(miembro_df['Palabras']))/miembro_df.shape[0]
    lista.append(palabras_por_msj)

    # Agregar a la lista el número de mensajes multimedia enviados
    multimedia = multimedia_df[multimedia_df['Miembro'] == miembros[i]].shape[0]
    lista.append(multimedia)

    # Agregar a la lista el número total de emojis enviados
    emojis = sum(miembro_df['Emojis'].str.len())
    lista.append(emojis)

    # Agregar a la lista el número total de links enviados
    links = sum(miembro_df['links'])
    lista.append(links)

    # Asignar la lista como valor a la llave del diccionario
    dictionario[miembros[i]] = lista
    
print(dictionario)

In [ ]:
# Convertir de diccionario a dataframe
miembro_stats_df = pd.DataFrame.from_dict(dictionario)

# Cambiar el índice por la columna agregada 'Estadísticas'
estadísticas = ['Mensajes', 'Palabras por mensaje', 'Multimedia', 'Emojis', 'Links']
miembro_stats_df['Estadísticas'] = estadísticas
miembro_stats_df.set_index('Estadísticas', inplace=True)

# Transponer el dataframe
miembro_stats_df = miembro_stats_df.T

#Convertir a integer las columnas Mensajes, Multimedia Emojis y Links
miembro_stats_df['Mensajes'] = miembro_stats_df['Mensajes'].apply(int)
miembro_stats_df['Multimedia'] = miembro_stats_df['Multimedia'].apply(int)
miembro_stats_df['Emojis'] = miembro_stats_df['Emojis'].apply(int)
miembro_stats_df['Links'] = miembro_stats_df['Links'].apply(int)
miembro_stats_df.sort_values(by=['Mensajes'], ascending=False)

# COMPORTAMIENTO DEL CHAT

In [ ]:
df['Hora'] = df['Hora'].str.replace('p. m.', 'PM').str.replace('a. m.', 'AM')
df['rangoHora'] = pd.to_datetime(df['Hora'])

# Define a function to create the "Range Hour" column
def create_range_hour(hour):
    start_hour = hour.hour
    end_hour = (hour + pd.Timedelta(hours=1)).hour
    return f'{start_hour:02d} - {end_hour:02d} h'

# # Apply the function to create the "Range Hour" column
df['rangoHora'] = df['rangoHora'].apply(create_range_hour)
df

# MENSAJES POR HORA

In [ ]:
# Crear una columna de 1 para realizar el conteo de mensajes
df['# Mensajes por hora'] = 1

df['# Mensajes por hora'] = df.groupby('rangoHora')['# Mensajes por hora'].transform('sum')

# Eliminar duplicados para evitar la sobrecontabilización
#df = df.drop_duplicates(subset=['rangoHora', '# Mensajes por hora'])
df_rangos = df.drop_duplicates(subset=['rangoHora', '# Mensajes por hora'])

#Plotear la cantidad de mensajes respecto del tiempo
plt.figure(figsize=(12, 6)) 
plt.plot(df_rangos['rangoHora'], df_rangos['# Mensajes por hora'], marker='o', linestyle='-', color='b')

# Personaliza el gráfico según tus preferencias
plt.title('Relación entre Rango de Hora y Cantidad de Mensajes')
plt.xlabel('Rango de Hora')
plt.ylabel('Cantidad de Mensajes')
plt.grid(True)

plt.figure(figsize=(24, 12)) 
plt.show()

In [ ]:
df_rangos

In [ ]:
df

# MENSAJES POR DIA

In [ ]:
# Crear una columna de 1 para realizar el conteo de mensajes
df['# Mensajes por dia'] = 1

# Sumar (contar) los mensajes que tengan la misma fecha
df['# Mensajes por dia'] = df.groupby('Fecha')['# Mensajes por dia'].transform('sum')

# Eliminar duplicados para evitar la sobrecontabilización
#df = df.drop_duplicates(subset=['rangoHora', '# Mensajes por hora'])
df_dias = df.drop_duplicates(subset=['Fecha', '# Mensajes por dia'])

df_dias

In [ ]:
plt.figure(figsize=(12, 6)) 
plt.plot(df_dias['Fecha'], df_dias['# Mensajes por dia'])

# Personaliza el gráfico según tus preferencias
plt.title('Relación entre Rango de Hora y Cantidad de Mensajes')
plt.xlabel('Rango de Hora')
plt.ylabel('Cantidad de Mensajes')
plt.grid(True)

plt.figure(figsize=(12, 6)) 
plt.show()

# PALABRAS QUE MAS USAMOS

In [ ]:
# Crear un string que contendrá todas las palabras
total_palabras = ' '

# Obtener y acumular todas las palabras de cada mensaje
for mensaje in mensajes_df['Mensaje'].values:
    palabras = str(mensaje).lower().split() # Obtener las palabras de cada línea del txt
    for palabra in palabras:
        total_palabras = total_palabras + palabra + ' ' # Acumular todas las palabras

wordcloud = WordCloud(width = 1000, height = 700,
                background_color ='white',
                stopwords = 'stopwords',
                min_font_size = 10).generate(total_palabras)

# Plotear la nube de palabras más usadas
wordcloud.to_image()